# Project Overview

In this project, we will build a market simulator that accepts trading orders while monitoring the value and performance of the portfolio.

## Our Imports

In [121]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

## Current Portfolio

Since our current portfolio is empty since no orders have been processed (and because the functionality is not complete), we should establish the data type or even just the structure of our portfolio. I've been playing with panda dataframes lately so let's go down that route.

In [122]:
portfolioColumns = ['Date','Symbol','Order','Shares','Price']
currentPortfolio = pd.DataFrame(columns=portfolioColumns)

In [123]:
currentPortfolio

,Date,Symbol,Order,Shares,Price


## Order Processing

Now when it comes to executing orders, it is a tri-part problem. We first need to fetch the price of a chosen stock. We then have to decide if this is an acceptable price to purchase or sell the stock. Finally, we have to execute this order. So let's begin.

### Fetch Stock Price

Let's leverage the yfinance python package to fetch a stock's particular price. For simplicity purposes, we will be getting the price of a chosen stock from it's adjusted close price for the day. In the future, I'm sure functionality can be extended to be more precise and allow for better stock price selection throughout the day.

In [124]:
def fetchPrice(stockChoices,inputDate):
    priceData = yf.download(stocks, start=inputDate, end=inputDate)
    return priceData
    
stockChoices = ['AAPL','GOOGL']

In [125]:
stockPrices = (fetchPrice(stockChoices,"2019-05-29")['Adj Close'])

[*********************100%***********************]  2 of 2 downloaded


Now that we have the the stockPrices in a nice pandas dataframe, all we have to do to filter it is the below.

In [160]:
for x in range(len(stockChoices)):
    print (stockPrices[stockChoices[x]][0])

177.38
1119.94


### Verify Order Exection

Here we want to decide if the price is an acceptable price to purchase/sell the stock. As well as how many units we want to purchase or sell. For now, we will have this as a manual method exercised by the user with functionality added to extend this to apply machine learning techniques to optimize it.

In [256]:
def decideOrder(stockPrice,stockUnits,userControl=False,tradeChoice="NA"):
    if (userControl == True):
        return (True,tradeChoice)
    else:
        return False 

### Execute Order

Here we will actually execute the order and update our portfolio and our current holdings. So let's work on an execute order function and a method to update our portfolio.

Here we want to work on an updatePortfolio function to reflect our executed orders impacts on our portfolio. 

In [267]:
def updatePortfolio(currPortfolio,addData):
    updatedPortfolio = currPortfolio.append(addData)
    return updatedPortfolio

Our execute order function takes in the following as input:
- stockChoices: Which stocks are to be part of this order
- stockUnits: How many of these stocks are to be traded
- inputDate: For which date is this order being made
- currPortfolio: Our currentPorfolio
- userControl: Whether to allow the user to execute or the machine learning model
- tradeChoice: Whether to buy, sell or hold an order

It will then return the boolean of the function and the updated portfolio.

In [268]:
def execOrder(stockChoices,stockUnits,inputDate,currPortfolio,userControl=False,tradeChoice="NA"):
    stockPrices = (fetchPrice(stockChoices,inputDate)['Adj Close'])
    orderDecision = decideOrder(stockPrices,stockUnits,userControl,tradeChoice)
    
    if (orderDecision[0] == True):
        for x in range(len(stockChoices)):
            addData = [(inputDate,stockChoices[x],orderDecision[1],stockUnits[x],stockPrices[stockChoices[x]][0])]
            addDataFrame = pd.DataFrame(addData,columns=currPortfolio.columns)
            currPortfolio = updatePortfolio(currPortfolio,addDataFrame)
        return (True,currPortfolio)
    else:
        return (False,currPortfolio)

Now if the order executed, I'd like to see my portfolio value. So let's create a display portfolio function to see my current portfolio value. An important consideration to remember is the fact that we may have multiple orders for the same stock. So we have to tally up and do a calculation on how many shares we still own and how much they're worth. 

In [317]:
def getPortfolioValue(currPortfolio):
    return portfolioValue

We should also have a function to get our starting investment to compare to the current portfolio value. This should be straightforward in the sense we find the very first order executed and calculate how much was invested from the number of shares and share price.

In [ ]:
def getStartInvestment(currPortfolio):
    return initialInvestment

In [314]:
def displayPortfolioValue(currPortfolio):
    totalValue = getPortfolioValue(currPortfolio)
    startValue = getStartInvestment(currPortfolio)
    
    print ("Welcome To Your Portfolio!\n")
    print ("Current holdings include: {0}".format(currPortfolio[1].Symbol.tolist()))
    print ("Current units held are: {0}".format(currPortfolio[1].Shares.tolist()))
    print ("Your initial investment was: {0}".format(startValue))
    print ("Total Portfolio Value: {0}".format(totalValue))

Now let's run a sample to test all this together.

In [335]:
updatedPortfolio = execOrder(['AAPL','GOOGL'],[10,10],"2019-05-01",currentPortfolio)

[*********************100%***********************]  2 of 2 downloaded


In [336]:
if (updatedPortfolio[0] == True):
    print ("Executed Order!\n")
    print ("Here is your trade history:\n")
    print (updatedPortfolio[1])
    print ("\n")
    displayPortfolioValue(updatedPortfolio)
else:
    print ("Order did not execute!\n")
    print ("Here is your trade history:\n")
    print (updatedPortfolio[1])

Executed Order!

Here is your trade history:

         Date Symbol Order Shares    Price
0  2019-05-01   AAPL   BUY     10   209.71
0  2019-05-01  GOOGL   BUY     10  1173.32


Welcome To Your Portfolio!

Current holdings include: ['AAPL', 'GOOGL']
Current units held are: [10, 10]
Total Portfolio Value: 13830.3


## Daily Portfolio Value

Now we want to see our portfolio value over a given time frame. So let's work on a function to do just that. This will leverage our portfolio calculation function that we worked on in the portfolio evaluator and optimizer projects.

In [332]:
def dailyPortfolioValue(currPortfolio,endDate,startDate=(updatedPortfolio[1].Date.tolist())[0]):
    dailyValues = yf.download(currPortfolio[1].Symbol.tolist(), start=startDate, end=endDate)
    normValues = dailyValues['Adj Close'] / dailyValues['Adj Close'].iloc[0]
    allocedValues = normValues * currPortfolio[1].Shares.tolist()
    posVals = allocedValues * getStartInvestment(currPortfolio)
    portVal = pd.DataFrame(get_portfolio_value(pos_vals),columns=['Value'])
    return portVal

Now let's test it out to see what our daily portfolio value is from the first order date to present day.

In [ ]:
portfolioValue = dailyPortfolioValue(updatedPortfolio,'2019-05-29')

## Assess Portfolio Performance

## Accounting for All Costs 

In this section we will tackle the real world costs associated with any order. We'll account for an initial investment as our starting cash balance. This is to check if we have enough funds to purchase the shares we need. On the other hand, we will also account for commission against the cash balance for each order executed. 

## Implementing Order Impact

In this section, how orders impact the market by using random orders to drive the price up/down. 

## Advancing Order Processing

Now let's add functionality to allow for market, limit, stop orders and maybe even short positions.